First we need to import our trainset along with all the libraries:

In [9]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D
from tensorflow.compat.v2.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers


trainingSet = 'trainSet'
testSet = 'testSet'

print("Tensorflow version %s" %tf.version)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Tensorflow version <module 'tensorflow_core._api.v2.version' from '/Users/giulioserra/.local/lib/python3.7/site-packages/tensorflow_core/_api/v2/version/__init__.py'>


SystemError: GPU device not found

Then we need to uniform all the images to the same size and resolutions in order to better train our model:

In [4]:
batch_size = 32

trainDatagen = ImageDataGenerator(
    rescale = 1. / 255,\
    zoom_range=0.1,\
    rotation_range=10,\
    width_shift_range=0.1,\
    height_shift_range=0.1,\
    horizontal_flip=True,\
    vertical_flip=False)



Now we need to provide the parameters to classify our samples, since we need to recognize immages by category we use class_mode by category:

In [5]:
trainGenerator = trainDatagen.flow_from_directory(
    directory=trainingSet,
    target_size=(256,256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)


samples = trainGenerator.n
classes = trainGenerator.num_classes
inputShape = trainGenerator.image_shape

classnames = [k for k,v in trainGenerator.class_indices.items()]

print("Matrix dimension: %s" %str(inputShape))
print("Classes found: %r" %classnames)

print('Loaded %d training samples from %d classes.' %(samples,classes))





Found 2000 images belonging to 4 classes.
Matrix dimension: (256, 256, 3)
Classes found: ['HAZE', 'RAINY', 'SNOWY', 'SUNNY']
Loaded 2000 training samples from 4 classes.


Now we need to load our testSet to validate our learning model(NOTE, the test wa adjusted to use the same name/number of categories of the trainSet ):

In [6]:
testDatagen = ImageDataGenerator(
    rescale = 1. / 255,\
    zoom_range=0.1,\
    rotation_range=10,\
    width_shift_range=0.1,\
    height_shift_range=0.1,\
    horizontal_flip=True,\
    vertical_flip=False)

testGenerator = testDatagen.flow_from_directory(
    directory=testSet,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

Tsamples = trainGenerator.n
Tclasses = trainGenerator.num_classes
TinputShape = trainGenerator.image_shape

Tclassnames = [k for k,v in testGenerator.class_indices.items()]

print("Matrix dimension: %s" %str(TinputShape))
print("Classes found: %r" %Tclassnames)

print('Loaded %d training samples from %d classes.' %(Tsamples,Tclasses))


Found 3038 images belonging to 4 classes.
Matrix dimension: (256, 256, 3)
Classes found: ['HAZE', 'RAINY', 'SNOWY', 'SUNNY']
Loaded 2000 training samples from 4 classes.


Now we need to create our model: a convolution neural network using AlexNet, then we train it with the loaded immages:

In [7]:
def AlexNet(input_shape, num_classes, regl2 = 0.0001, lr=0.0001):
    
    model = Sequential()

    # C1 Convolutional Layer 
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11),\
                     strides=(2,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # C2 Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C3 Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C4 Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # C5 Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Flatten
    model.add(Flatten())

    flatten_shape = (input_shape[0]*input_shape[1]*input_shape[2],)
    
    # D1 Dense Layer
    model.add(Dense(4096, input_shape=flatten_shape, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # D2 Dense Layer
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # D3 Dense Layer
    model.add(Dense(1000,kernel_regularizer=regularizers.l2(regl2)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # Compile

    adam = optimizers.Adam(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model
 
# create the model
model = AlexNet(inputShape,classes)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 62, 96)       34944     
_________________________________________________________________
activation (Activation)      (None, 123, 62, 96)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 31, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 61, 31, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 51, 21, 256)       2973952   
_________________________________________________________________
activation_1 (Activation)    (None, 51, 21, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 10, 256)       0

Now we have to train our model while validating it with the test set:

In [8]:
start_time = time.time()

steps_per_epoch=trainGenerator.n//trainGenerator.batch_size
val_steps=testGenerator.n//testGenerator.batch_size+1


history = model.fit_generator(trainGenerator, epochs=100, verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=testGenerator,\
                    validation_steps=val_steps)


print("-------------------------------------------")
elapsed_time = time.time() - start_time
    
print("elapsed Time: %.3fs" %elapsed_time)
print("-------------------------------------------")



Epoch 1/100
 3/62 [>.............................] - ETA: 5:48 - loss: 2.9023 - accuracy: 0.3125

/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag)
/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)


12/62 [====>.........................] - ETA: 4:52 - loss: 2.5044 - accuracy: 0.4792

/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag)
/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)


28/62 [============>.................] - ETA: 3:14 - loss: 2.3562 - accuracy: 0.5234

...

/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag)
/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:784: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)


39/62 [=================>............] - ETA: 2:37 - loss: 1.5502 - accuracy: 0.7887

/opt/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


61/62 [============================>.] - ETA: 7s - loss: 1.5537 - accuracy: 0.7896 

KeyboardInterrupt: 

Now we have to plot the classification report to evaluate our model data:

In [ ]:
val_steps=testGenerator.n//testGenerator.batch_size+1

preds = model.predict_generator(testGenerator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = testGenerator.classes  # shuffle=False in test_generator

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

cm = confusion_matrix(Ytest, Ypred)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')

for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))
  

Now we can plot our results:

In [ ]:
for i in history.history :
  print(i)
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()